# Rust-Python Binding Notebook Course

![](https://hackmd.io/_uploads/S1qiK8hRh.png)
> Image from [WebAssembly Interface Types: Interoperate with All the Things!](https://hacks.mozilla.org/2019/08/webassembly-interface-types/)


In [1]:
%pip install -r requirements.txt
!maturin develop --release


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /home/peter/miniconda3/bin/python
📡 Using build options features from pyproject.toml
   Compiling autocfg v1.1.0
   Compiling libc v0.2.147
   Compiling cfg-if v1.0.0
   Compiling proc-macro2 v1.0.66
   Compiling unicode-ident v1.0.11
   Compiling version_check v0.9.4
   Compiling once_cell v1.18.0
   Compiling bytes v1.5.0
   Compiling pin-project-lite v0.2.13
   Compiling futures-core v0.3.28
   Compiling memchr v2.6.3
   Compiling value-bag v1.4.1
   Compiling futures-io v0.3.28
   Compiling libm v0.2.7
   Compiling scopeguard v1.2.0
   Compiling typenum v1.16.0
   Compiling itoa v1.0.9
   Compiling parking_lot_core v0.9.8
   Compiling pkg-config v0.3.27
   Compiling smallvec v1.11.0
   Compiling futures-sink v0.3.28
   Compiling pin-utils v0.1.0
   Compiling vc

In [2]:
import blogpost_ffi
import os
import pyarrow as pa
import time
import tracing
from opentelemetry.trace.propagation.tracecontext import (
    TraceContextTextMapPropagator,
)

In [3]:
pa.array([])
value = [1] * 100_000_000

In [4]:
# Default implementation

print(f"---Default Implementation---")

start_time = time.time()

array = blogpost_ffi.create_list(value)

print(f"default: {time.time() - start_time:.3f}s")

---Default Implementation---
default: 2.616s


In [5]:
# PyBytes Implementation

print(f"---PyBytes Implementation---")

value_bytes = bytes(value)

start_time = time.time()

array = blogpost_ffi.create_list_bytes(value_bytes)

print(f"bytes: {time.time() - start_time:.3f}s")


---PyBytes Implementation---
bytes: 0.125s


In [6]:
# Arrow Implementation

print(f"---Arrow Implementation---")

value_arrow = pa.array(value, type=pa.uint8())

start_time = time.time()

array = blogpost_ffi.create_list_arrow(value_arrow)

print(f"arrow: {time.time() - start_time:.3f}s")

---Arrow Implementation---
arrow: 0.049s


In [7]:
# Debugging eyre

print(f"---Eyre error---")

# Try:
# array = blogpost_ffi.create_list_arrow(1)
#
## This error panics the whole program and is therefore uncatchable.
ERROR_WITHOUT_EYRE = """
thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }', src/lib.rs:45:62
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Traceback (most recent call last):
  File "/home/peter/Documents/work/blogpost_ffi/test_script.py", line 79, in <module>
    array = blogpost_ffi.create_list_arrow(1)
pyo3_runtime.PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }
"""


# Eyre result
array = blogpost_ffi.create_list_arrow_eyre(1)

---Eyre error---


RuntimeError: Could not convert arrow data

Caused by:
    TypeError: Expected instance of pyarrow.lib.Array, got builtins.int

Location:
    src/lib.rs:73:50

In [8]:
# Eyre default

def abc():
    assert False, "I have no idea what is wrong"

In [9]:
print(f"---Eyre no traceback---")

array = blogpost_ffi.call_func_eyre(abc)

---Eyre no traceback---


RuntimeError: function called failed

Caused by:
    AssertionError: I have no idea what is wrong

Location:
    src/lib.rs:102:39

In [10]:
# Eyre traceback
print(f"---Eyre traceback---")

array = blogpost_ffi.call_func_eyre_traceback(abc)

---Eyre traceback---


RuntimeError: function called failed

Caused by:
    Traceback (most recent call last):
      File "/tmp/ipykernel_192938/1312320087.py", line 4, in abc

    AssertionError: I have no idea what is wrong

Location:
    src/lib.rs:109:9

In [11]:
# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.unbounded_memory_growth()

-->Open a Memory analyzer


In [12]:

# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.bounded_memory_growth()

-->Open a Memory analyzer


## The GIL

```
`Python` exists   |=====================================|
GIL actually held |==========|         |================|
Rust code running |=======|                |==|  |======|
```


This behaviour can cause deadlocks when trying to lock a Rust mutex while holding the GIL:

Thread 1 acquires the GIL
Thread 1 locks a mutex
Thread 1 makes a call into the Python interpreter which releases the GIL
Thread 2 acquires the GIL
Thread 2 tries to locks the mutex, blocks
Thread 1’s Python interpreter call blocks trying to reacquire the GIL held by thread 2
To avoid deadlocking, you should release the GIL before trying to lock a mutex or awaiting in asynchronous code, e.g. with Python::allow_threads.

Releasing and freeing memory


See: https://github.com/PyO3/pyo3/issues/576 for sub-interpreter that can remove some of the GIL Issues
https://github.com/Aequitosh/pyo3/discussions/1 


In [13]:
# GIL Lock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_lock()

---GIL Lock---
This threaded print was printed after 10.000256766s


In [14]:
# GIL unlock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_unlock()

---GIL Lock---
1. This was printed after 34.945µs
2. This was printed after 284.441µs


In [15]:
!docker run -d -p6831:6831/udp -p6832:6832/udp -p16686:16686 jaegertracing/all-in-one:latest

309d0a6e7e563a4a08ae8189fea10d1a60575567a69df1dc0574696016f40bdf
docker: Error response from daemon: driver failed programming external connectivity on endpoint busy_brattain (3de390d50682d856e85d8c0725ec78fc2bc5675e8f7dbd8f3ab32f1611fc3e4f): Bind for 0.0.0.0:16686 failed: port is already allocated.


In [16]:
# Global tracing
print(f"---Global tracing---")

def abc(cx):
    propagator = TraceContextTextMapPropagator()
    context = propagator.extract(carrier=cx)

    with tracing.tracer.start_as_current_span(
        name="Python_span", context=context
    ) as child_span:
        child_span.add_event("in Python!")
        output = {}
        tracing.propagator.inject(output)
        time.sleep(2)
    return output


array = blogpost_ffi.global_tracing(abc)

---Global tracing---


In [17]:
# Distributing the code

!maturin publish --non-interactive

🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /home/peter/miniconda3/bin/python3
📡 Using build options features from pyproject.toml
   Compiling pyo3-build-config v0.19.2
   Compiling pyo3-ffi v0.19.2========> ] 422/431: pyo3-build-config           
   Compiling pyo3 v0.19.2
   Compiling arrow v46.0.0========> ] 428/431: pyo3                           
   Compiling blogpost_ffi v0.1.0 (/home/peter/Documents/work/blogpost_ffi)    
    Finished release [optimized] target(s) in 3.15slogpost_ffi                
🖨  Copied external shared libraries to package blogpost_ffi.libs directory:
    /usr/lib/x86_64-linux-gnu/libp11-kit.so.0.3.0
    /usr/lib/x86_64-linux-gnu/libpsl.so.5.3.4
    /usr/lib/x86_64-linux-gnu/libkrb5.so.3.3
    /usr/lib/x86_64-linux-gnu/libtasn1.so.6.6.3
    /usr/lib/x86_64-linux-gnu/libsasl2.so.2.0.25
    /usr/lib/x86_64-linux-gnu/libgssapi_krb5.so.2.2
    /usr/lib/x86_64-linux-gnu/libkeyutils.so.1.10
    /usr/lib/x86_64-linux-gnu/libbrotlicommon.so.1.0.9
    /usr/lib/x86

In [4]:
!maturin  generate-ci github --zig --platform all                                                       

🔗 Found pyo3 bindings
# This file is autogenerated by maturin v1.2.3
# To update, run
#
#    maturin generate-ci github --zig --platform all
#
name: CI

on:
  push:
    branches:
      - main
      - master
    tags:
      - '*'
  pull_request:
  workflow_dispatch:

permissions:
  contents: read

jobs:
  linux:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        target: [x86_64, x86, aarch64, armv7, s390x, ppc64le]
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Build wheels
        uses: PyO3/maturin-action@v1
        with:
          target: ${{ matrix.target }}
          args: --release --out dist --find-interpreter --zig
          sccache: 'true'
          manylinux: auto
      - name: Upload wheels
        uses: actions/upload-artifact@v3
        with:
          name: wheels
          path: dist

  windows:
    runs-on: windows-latest
    strategy:
      matrix:
        target

## Rust-Python Parallelism

See: https://pyo3.rs/v0.14.3/parallelism


## Rust-Python Async

This is based on a double queue. One managed by rust/async and the other by python/asyncio and a system of waker/caller. 

Issues is that this makes the system most of the time less performant than with a sync and tokio channel in our test, except if you have a very large number of await points in Python.

This is specially not performant when we're doing CPU-bound tasks.

See: https://docs.rs/pyo3-asyncio/latest/pyo3_asyncio/

